# ICIJ analysis

## Set up

Load the Python dependencies.

In [1]:
from collections import Counter
from dataclasses import dataclass, field
import json
import os
import pathlib
import re
import sys
import typing

from icecream import ic
from tqdm import tqdm
import kuzu
import pandas as pd
import watermark

%load_ext watermark

In [2]:
%watermark
%watermark --iversions

Last updated: 2024-07-07T19:23:05.161405-07:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.26.0

Compiler    : Clang 13.0.0 (clang-1300.0.29.30)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

kuzu     : 0.4.2
sys      : 3.11.9 (v3.11.9:de54cf5be3, Apr  2 2024, 07:12:50) [Clang 13.0.0 (clang-1300.0.29.30)]
json     : 2.0.9
watermark: 2.4.3
pandas   : 2.2.2
re       : 2.2.1



Establish a connection to KùzuDB

In [3]:
!rm -rf ./demo

In [4]:
db: kuzu.database.Database = kuzu.Database("./demo")
conn: kuzu.database.Database = kuzu.Connection(db)

## Data discovery

In [5]:
dat_dir: pathlib.Path = pathlib.Path("full_2023")

### Entities

In [6]:
dat_file: pathlib.Path = dat_dir / "nodes-entities.csv"

df = pd.read_csv(
    dat_file,
    header = 0,
    low_memory = False,
).astype(str).fillna("")

df.replace({"nan": ""}, regex=True, inplace=True)

df.head()

,node_id,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,incorporation_date,...,struck_off_date,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note
0,10000001,"TIANSHENG INDUSTRY AND TRADING CO., LTD.","TIANSHENG INDUSTRY AND TRADING CO., LTD.",,SAM,Samoa,,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1001256.0,23-MAR-2006,...,15-FEB-2013,,Defaulted,Mossack Fonseca,25221,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,
1,10000002,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.","NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",,SAM,Samoa,,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1001263.0,27-MAR-2006,...,15-FEB-2014,,Defaulted,Mossack Fonseca,25249,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,
2,10000003,"HOTFOCUS CO., LTD.","HOTFOCUS CO., LTD.",,SAM,Samoa,,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000896.0,10-JAN-2006,...,15-FEB-2012,,Defaulted,Mossack Fonseca,24138,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,
3,10000004,"SKY-BLUE GIFTS & TOYS CO., LTD.","SKY-BLUE GIFTS & TOYS CO., LTD.",,SAM,Samoa,,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000914.0,06-JAN-2006,...,15-FEB-2009,,Defaulted,Mossack Fonseca,24012,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,
4,10000005,FORTUNEMAKER INVESTMENTS CORPORATION,FORTUNEMAKER INVESTMENTS CORPORATION,,SAM,Samoa,,LOYAL PORT LIMITED 8/F; CRAWFORD TOWER 99 JERV...,1001266.0,19-APR-2006,...,15-FEB-2008,,Changed agent,Mossack Fonseca,R25638,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,


In [7]:
df.describe(include = "all").loc[[ "count", "freq", "unique", "top", ]]

,node_id,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,incorporation_date,...,struck_off_date,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note
count,814344,814344,814344,814344,814344,814344,814344,814344,814344,814344,...,814344,814344,814344,814344,814344,814344,814344,814344,814344,814344
freq,1,29,424822,807507,209634,209713,675593,515021,424822,25874,...,470501,794137,456310,470258,251878,309353,309353,213634,213634,772569
unique,814344,781569,368821,6783,103,88,66,20583,343795,19009,...,9083,276,86,5,511469,1102,1103,22,65,207
top,10000001,,,,BAH,Bahamas,,,,,...,,,,,,,,Panama Papers,The Panama Papers data is current through 2015,


What are the `type` field values?

In [8]:
set(df.company_type.values)

{'',
 'Antigua',
 'Audit Licence',
 'BVI Share Trust',
 'BVI Sundry Entities (one off transactions)',
 'BVI Trust',
 'Bahamas IBC',
 'Belize International Business Company',
 'Busines Company Limited by Shares & Guarantee',
 'Business Company Limited by Guarantee',
 'Business Company Limited by Shares',
 'Business Company Restricted Purposes',
 'Business Corporation',
 'Business Vehicle',
 'CAP 285',
 'Client Sundry Account',
 'Collective Investment Scheme',
 'Cook Islands Asset Protection Trust',
 'Cook Islands Asset Protection Trust - 3520A',
 'Cook Islands Trust',
 'Domestic Company',
 'EXEMPT INSURANCE HOLDING REGISTER',
 'EXEMPT INSURANCE REGISTER',
 'Foreign Company',
 'Foreign Company Transfer',
 'Holding Company',
 'Hong Kong',
 'Hong Kong Trust',
 'International Business Corporation',
 'International Company',
 'International Trust',
 'Limited Liability Company',
 'Limited Partnership',
 'Liquidator Licence',
 'Mail Forwarding Only',
 'Mauritius - Hybrid',
 'Mauritius - Intern

Keep track of the set of `node_id` values for this class.

In [9]:
id_entity: set = set(df.node_id.values)

After iterating the first time through this analysis, we return to this point and redefine an `Entity` node structure which is a superset of the fields defined among all of the Entity-ish nodes in ICIJ.
See <https://docs.google.com/spreadsheets/d/1eSelhXhix_DtTZuzR2vfl_UdQlEwbwql6NZxqrtROxk/edit?usp=sharing>

In [10]:
conn.execute("""
  CREATE NODE TABLE Entity (
    node_id STRING,
    role STRING,
    name STRING,
    original_name STRING,
    former_name STRING,
    jurisdiction STRING,
    jurisdiction_description STRING,
    company_type STRING,
    address STRING,
    internal_id STRING,
    incorporation_date STRING,
    inactivation_date STRING,
    struck_off_date STRING,
    dorm_date STRING,
    status STRING,
    service_provider STRING,
    ibcRUC STRING,
    country_codes STRING,
    countries STRING,
    sourceID STRING,
    valid_until STRING,
    note STRING,
    vague BOOLEAN,
    PRIMARY KEY (node_id)
  )
""");

Reshape the dataframe to fit our inclusive `Entity` metadata.

In [11]:
df.columns.values.tolist()

['node_id',
 'name',
 'original_name',
 'former_name',
 'jurisdiction',
 'jurisdiction_description',
 'company_type',
 'address',
 'internal_id',
 'incorporation_date',
 'inactivation_date',
 'struck_off_date',
 'dorm_date',
 'status',
 'service_provider',
 'ibcRUC',
 'country_codes',
 'countries',
 'sourceID',
 'valid_until',
 'note']

In [12]:
df.insert(1, "role", "Entity")

df["vague"] = False

df.head()

,node_id,role,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,...,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note,vague
0,10000001,Entity,"TIANSHENG INDUSTRY AND TRADING CO., LTD.","TIANSHENG INDUSTRY AND TRADING CO., LTD.",,SAM,Samoa,,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1001256.0,...,,Defaulted,Mossack Fonseca,25221,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,,False
1,10000002,Entity,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.","NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",,SAM,Samoa,,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1001263.0,...,,Defaulted,Mossack Fonseca,25249,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,,False
2,10000003,Entity,"HOTFOCUS CO., LTD.","HOTFOCUS CO., LTD.",,SAM,Samoa,,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000896.0,...,,Defaulted,Mossack Fonseca,24138,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,,False
3,10000004,Entity,"SKY-BLUE GIFTS & TOYS CO., LTD.","SKY-BLUE GIFTS & TOYS CO., LTD.",,SAM,Samoa,,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000914.0,...,,Defaulted,Mossack Fonseca,24012,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,,False
4,10000005,Entity,FORTUNEMAKER INVESTMENTS CORPORATION,FORTUNEMAKER INVESTMENTS CORPORATION,,SAM,Samoa,,LOYAL PORT LIMITED 8/F; CRAWFORD TOWER 99 JERV...,1001266.0,...,,Changed agent,Mossack Fonseca,R25638,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,,False


In [13]:
conn.execute(
    "COPY Entity FROM (LOAD FROM df RETURN *)"
);

In [14]:
results = conn.execute("""
  MATCH (n:Entity)
  RETURN *
  LIMIT 1;
""")

while results.has_next():
    ic(results.get_next())

ic| results.get_next(): [{'_id': {'offset': 20480, 'table': 0},
                          '_label': 'Entity',
                          'address': '',
                          'company_type': '',
                          'countries': '',
                          'country_codes': '',
                          'dorm_date': '',
                          'former_name': '',
                          'ibcRUC': '93990B',
                          'inactivation_date': '',
                          'incorporation_date': '03-AUG-1999',
                          'internal_id': '93990.0',
                          'jurisdiction': 'BAH',
                          'jurisdiction_description': 'Bahamas',
                          'name': 'RICHFIELD CONSULTANCY SERVICES LIMITED',
                          'node_id': '20093990',
                          'note': '',
                          'original_name': 'RICHFIELD CONSULTANCY SERVICES LIMITED',
                          'role': 'Entity',
       

In [15]:
results = conn.execute("""
  MATCH (n:Entity)
  RETURN COUNT(*)
""")

while results.has_next():
    ic(results.get_next())

ic| results.get_next(): [814344]


### Officers

In [16]:
dat_file: pathlib.Path = dat_dir / "nodes-officers.csv"

df = pd.read_csv(
    dat_file,
    header = 0,
    low_memory = False,
).astype(str).fillna("")

df.replace({"nan": ""}, regex=True, inplace=True)

df.head()

,node_id,name,countries,country_codes,sourceID,valid_until,note
0,12000001,KIM SOO IN,South Korea,KOR,Panama Papers,The Panama Papers data is current through 2015,
1,12000002,Tian Yuan,China,CHN,Panama Papers,The Panama Papers data is current through 2015,
2,12000003,GREGORY JOHN SOLOMON,Australia,AUS,Panama Papers,The Panama Papers data is current through 2015,
3,12000004,MATSUDA MASUMI,Japan,JPN,Panama Papers,The Panama Papers data is current through 2015,
4,12000005,HO THUY NGA,Viet Nam,VNM,Panama Papers,The Panama Papers data is current through 2015,


In [17]:
df.describe(include = "all").loc[[ "count", "freq", "unique", "top", ]]

,node_id,name,countries,country_codes,sourceID,valid_until,note
count,771315,771315,771315,771315,771315,771315,771315
freq,1,70873,302334,301057,238402,238402,767554
unique,771315,538360,4091,4772,22,81,24
top,12000001,THE BEARER,,,Panama Papers,The Panama Papers data is current through 2015,


Note: phrases such as "THE BEARER" or "EL PORTADOR" are contract placeholders for whomever holds the bearer shares, and we need to filter them from entity resolution or constructing relations.

In [18]:
names: dict = df["name"].value_counts().to_dict()
names

{'THE BEARER': 70873,
 'EL PORTADOR': 9325,
 'Bearer 1': 2655,
 'CARMICHAEL TREVOR A.': 1196,
 'CLEMENTI LIMITED': 1111,
 'TANAH MERAH LIMITED': 1046,
 'BUKIT MERAH LIMITED': 963,
 'CST ADMINISTRATION (BAHAM': 835,
 'Bearer': 818,
 'The Bearer': 813,
 'THE CORPORATE SECRETARY LIMITED': 700,
 'Christopher Marcus GRADEL': 576,
 'BEARER': 537,
 'COURT ADMINISTRATION LIMI': 474,
 'PRIMARY MANAGEMENT LIMITE': 449,
 'BARNES DEBORAH J.': 416,
 'FIELDS JAMES A.': 408,
 'BLUE SEAS ADMINISTRATION': 389,
 'STANDARD NOMINEES (BAHAMA': 384,
 "TRIDENT CORPORATE SERVICES (B'DOS) LTD": 369,
 'CIRCLE CORPORATE SERVICES': 343,
 'CORPORATE SERVICES LIMITED': 319,
 'GUARDIAN NOMINEES (BARBADOS) LIMITED': 288,
 'IL SHIN CORPORATE CONSULTING LIMITED': 286,
 'KHAN ZARINA': 283,
 'CASCADO AG': 275,
 'OCTAGON MANAGEMENT LIMITE': 272,
 'Vanessa Marie-Antoine PAYET': 259,
 'AMICORP (BARBADOS) LTD.': 243,
 'Shirley Sabia Therese VAN KERKHOVE': 242,
 'MANEX LIMITED': 238,
 'BDG MANAGEMENT LTD.': 231,
 'SISNETT NAT

In [19]:
suspect: dict = {
    name.lower(): count
    for name, count in names.items()
    if count >= 2 and ("bearer" in name.lower() or "portador" in name.lower())
}

suspect

{'the bearer': 4,
 'el portador': 25,
 'bearer 1': 13,
 'bearer': 16,
 'the  bearer': 114,
 'bearer 2': 10,
 'to the bearer': 6,
 'al portador': 39,
 'bearer 3': 4,
 'bearer 4': 5,
 'bearer 5': 2,
 'bearer 6': 16,
 'bearer02': 14,
 'bearer 8': 14,
 'bearer 7': 14,
 'to bearer': 14,
 'bearer 9': 12,
 'bearer  - bearer agent - liang chen yuen chi': 10,
 'bearer 10': 10,
 'bearer1': 6,
 'the bearer 50,000 shares': 7,
 'the bearer at 500 shares': 7,
 'the bearer (agent: mr. mak kin man)': 7,
 'bearer share': 5,
 'the bearer at 50,000 shares': 6,
 'the bearer of 50,000 shares': 6,
 'the bearer (agent: pro-corp nominees limited)': 6,
 'bearer: agent-wirja tanizar': 6,
 'bearer for immota foundation': 6,
 'bearer01': 2,
 'bearer.': 5,
 'bearer03': 5,
 'bearer 12': 5,
 'bearer (elena kyprianou)': 5,
 'bearer2': 4,
 'the bearer agent: wbc secretaries ltd.': 4,
 'the bearer agent: ann lay': 4,
 'the bearer1': 4,
 '-the bearer': 4,
 'bearer 14': 4,
 'bearer 13': 4,
 'bearer 15': 4,
 'bearer10': 4

In [20]:
len(suspect)

146

Identify regex patterns to use for filtering these legalese placeholders.

In [21]:
import re

PAT_LIST: list = [
    r"^\-?(to\s+)?([the]+\s+)?bearer\.?\s?(\d+)?(\w)?$",
    r"^.*bearer.*shares?$",
    r"^the\s+bearer\s+\([\d\,]+\)$",
    r"^[ae]l\s+portador$",
    r"^the\s?bearer$",
    r"^bearer\s?warrant$",
    r"^bearer\s?shareholder$",
    r"^the\,\s+bearer$",
    r"^bearer\s+\(reedeem\s+shares\)$",
    r"^the\s+bearer\s+\(lost\)$",
    r"^bearer\s+\-\s+[\w]$",
    r"^bearer\s+\"\w\"$",
    r"^bearer\s+[\d\-]+$",
    r"^bearer\s+no\.\s+\d+$",
    r"^the\s+bearer\s+at\s+[\d\,]+$",
    r"^nan$",
    r"^[\?]+$",
]

def filter_bearer (
    name: str,
    ) -> bool:
    name = name.lower()

    for pat in PAT_LIST:
        if re.search(pat, name) is not None:
            return False

    return True

total: int = 0

for name, count in suspect.items():
    if filter_bearer(name):
        ic(name, count)
        total += 1

total

ic| name: 'bearer  - bearer agent - liang chen yuen chi', count: 10
ic| name: 'the bearer (agent: mr. mak kin man)', count: 7
ic| name: 'the bearer (agent: pro-corp nominees limited)', count: 6
ic| name: 'bearer: agent-wirja tanizar', count: 6
ic| name: 'bearer for immota foundation', count: 6
ic| name: 'bearer (elena kyprianou)', count: 5
ic| name: 'the bearer agent: wbc secretaries ltd.', count: 4
ic| name: 'the bearer agent: ann lay', count: 4
ic| name: 'the bearer (mr. andrey komarov)', count: 4
ic| name: 'the bearer:agent davenhill assets limited', count: 4
ic| name: 'the bearer (mr. yuriy vasilievich schastlivyi)', count: 3
ic| name: 'the bearer agent-kan pui wai', count: 3
ic| name: 'bearer agent: toni juhani salmela', count: 3
ic| name: 'bearer (vinsonburg management corp.)', count: 3
ic| name: 'bearer agent:- pro-corp nominees limited', count: 3
ic| name: 'bearer pro-corp nominees limited', count: 3
ic| name: 'the bearer (agent pro-corp nominees limited', count: 3
ic| name: 't

57

Confirm whether this filter works…

In [22]:
counter: Counter = Counter()

for name, count in df["name"].value_counts().to_dict().items():
    if filter_bearer(name):
        counter[name] += count

counter.most_common(1000)

[('CARMICHAEL TREVOR A.', 1196),
 ('CLEMENTI LIMITED', 1111),
 ('TANAH MERAH LIMITED', 1046),
 ('BUKIT MERAH LIMITED', 963),
 ('CST ADMINISTRATION (BAHAM', 835),
 ('THE CORPORATE SECRETARY LIMITED', 700),
 ('Christopher Marcus GRADEL', 576),
 ('COURT ADMINISTRATION LIMI', 474),
 ('PRIMARY MANAGEMENT LIMITE', 449),
 ('BARNES DEBORAH J.', 416),
 ('FIELDS JAMES A.', 408),
 ('BLUE SEAS ADMINISTRATION', 389),
 ('STANDARD NOMINEES (BAHAMA', 384),
 ("TRIDENT CORPORATE SERVICES (B'DOS) LTD", 369),
 ('CIRCLE CORPORATE SERVICES', 343),
 ('CORPORATE SERVICES LIMITED', 319),
 ('GUARDIAN NOMINEES (BARBADOS) LIMITED', 288),
 ('IL SHIN CORPORATE CONSULTING LIMITED', 286),
 ('KHAN ZARINA', 283),
 ('CASCADO AG', 275),
 ('OCTAGON MANAGEMENT LIMITE', 272),
 ('Vanessa Marie-Antoine PAYET', 259),
 ('AMICORP (BARBADOS) LTD.', 243),
 ('Shirley Sabia Therese VAN KERKHOVE', 242),
 ('MANEX LIMITED', 238),
 ('BDG MANAGEMENT LTD.', 231),
 ('SISNETT NATALIA B.', 230),
 ('WORME ROBERT C.', 220),
 ('CALLAGHAN DAVID 

Now add a `vague` column to flag the dubious names

In [23]:
df["vague"] = df.apply(lambda x: not filter_bearer(x["name"]), axis = 1)

Keep track of the set of `node_id` values for this class.

In [24]:
id_officer: set = set(df[df.vague == False].node_id.values)

Reshape the dataframe to fit our inclusive `Entity` metadata.

In [25]:
df.columns.values.tolist()

['node_id',
 'name',
 'countries',
 'country_codes',
 'sourceID',
 'valid_until',
 'note',
 'vague']

In [26]:
df.insert(1, "role", "Officer")

df.insert(3, "original_name", "")
df.insert(4, "former_name", "")
df.insert(5, "jurisdiction", "")
df.insert(6, "jurisdiction_description", "")
df.insert(7, "company_type", "")
df.insert(8, "address", "")
df.insert(9, "internal_id", "")
df.insert(10, "incorporation_date", "")
df.insert(11, "inactivation_date", "")
df.insert(12, "struck_off_date", "")
df.insert(13, "dorm_date", "")
df.insert(14, "status", "")
df.insert(15, "service_provider", "")
df.insert(16, "ibcRUC", "")

In [27]:
columns: list = list(df.columns.values)

columns[17] = "country_codes"
columns[18] = "countries"

columns

['node_id',
 'role',
 'name',
 'original_name',
 'former_name',
 'jurisdiction',
 'jurisdiction_description',
 'company_type',
 'address',
 'internal_id',
 'incorporation_date',
 'inactivation_date',
 'struck_off_date',
 'dorm_date',
 'status',
 'service_provider',
 'ibcRUC',
 'country_codes',
 'countries',
 'sourceID',
 'valid_until',
 'note',
 'vague']

In [28]:
df = df.reindex(columns = columns)
df.head()

,node_id,role,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,...,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note,vague
0,12000001,Officer,KIM SOO IN,,,,,,,,...,,,,,KOR,South Korea,Panama Papers,The Panama Papers data is current through 2015,,False
1,12000002,Officer,Tian Yuan,,,,,,,,...,,,,,CHN,China,Panama Papers,The Panama Papers data is current through 2015,,False
2,12000003,Officer,GREGORY JOHN SOLOMON,,,,,,,,...,,,,,AUS,Australia,Panama Papers,The Panama Papers data is current through 2015,,False
3,12000004,Officer,MATSUDA MASUMI,,,,,,,,...,,,,,JPN,Japan,Panama Papers,The Panama Papers data is current through 2015,,False
4,12000005,Officer,HO THUY NGA,,,,,,,,...,,,,,VNM,Viet Nam,Panama Papers,The Panama Papers data is current through 2015,,False


In [29]:
conn.execute(
    "COPY Entity FROM (LOAD FROM df RETURN *)"
);

In [30]:
results = conn.execute("""
  MATCH (n:Entity)
  RETURN COUNT(*)
""")

while results.has_next():
    ic(results.get_next())

ic| results.get_next(): [1585659]


### Intermediaries

In [31]:
dat_file: pathlib.Path = dat_dir / "nodes-intermediaries.csv"

df = pd.read_csv(
    dat_file,
    header = 0,
    low_memory = False,
).astype(str).fillna("")

df.replace({"nan": ""}, regex=True, inplace=True)

df.head()

,node_id,name,status,internal_id,address,countries,country_codes,sourceID,valid_until,note
0,11000001,"MICHAEL PAPAGEORGE, MR.",ACTIVE,10001,MICHAEL PAPAGEORGE; MR. 106 NICHOLSON STREET B...,South Africa,ZAF,Panama Papers,The Panama Papers data is current through 2015,
1,11000002,CORFIDUCIA ANSTALT,ACTIVE,10004,,Liechtenstein,LIE,Panama Papers,The Panama Papers data is current through 2015,
2,11000003,"DAVID, RONALD",SUSPENDED,10014,,Monaco,MCO,Panama Papers,The Panama Papers data is current through 2015,
3,11000004,"DE BOUTSELIS, JEAN-PIERRE",SUSPENDED,10015,,Belgium,BEL,Panama Papers,The Panama Papers data is current through 2015,
4,11000005,THE LEVANT LAWYERS (TLL),ACTIVE,10029,,Lebanon,LBN,Panama Papers,The Panama Papers data is current through 2015,


In [32]:
df.describe(include = "all").loc[[ "count", "freq", "unique", "top", ]]

,node_id,name,status,internal_id,address,countries,country_codes,sourceID,valid_until,note
count,26768,26768,26768,26768,26768,26768,26768,26768,26768,26768
freq,1,62,14147,12117,18125,4895,4895,14110,14110,26761
unique,26768,25590,10,14282,8640,286,286,9,11,7
top,11000001,HUTCHINSON GAYLE A.,,,,Hong Kong,HKG,Panama Papers,The Panama Papers data is current through 2015,


The `Intermediaries` defined by ICIJ have loads of overlapping records. These need to be removed.

In [33]:
len(id_officer.intersection(set(df.node_id.values)))

1139

In [34]:
def check_row (
    node_id: int,
    ) -> bool:
    return node_id not in id_officer

df["keep"] = df.apply(lambda x: check_row(x["node_id"]), axis = 1)
df = df[df.keep]
del df["keep"]

len(id_officer.intersection(set(df.node_id.values)))

0

In [35]:
df.describe(include = "all").loc[[ "count", "freq", "unique", "top", ]]

,node_id,name,status,internal_id,address,countries,country_codes,sourceID,valid_until,note
count,25629,25629,25629,25629,25629,25629,25629,25629,25629,25629
freq,1,62,13008,10978,16986,4719,4719,14110,14110,25622
unique,25629,24463,10,14282,8640,279,279,9,11,7
top,11000001,HUTCHINSON GAYLE A.,,,,Hong Kong,HKG,Panama Papers,The Panama Papers data is current through 2015,


Keep track of the set of `node_id` values for this class.

In [36]:
id_intermed: set = set(df.node_id.values)

Reshape the dataframe to fit our inclusive `Entity` metadata.

In [37]:
df.columns.values.tolist()

['node_id',
 'name',
 'status',
 'internal_id',
 'address',
 'countries',
 'country_codes',
 'sourceID',
 'valid_until',
 'note']

In [38]:
df.insert(1, "role", "Intermediary")

df.insert(3, "original_name", "")
df.insert(4, "former_name", "")
df.insert(5, "jurisdiction", "")
df.insert(6, "jurisdiction_description", "")
df.insert(7, "company_type", "")

df.insert(11, "incorporation_date", "")
df.insert(12, "inactivation_date", "")
df.insert(13, "struck_off_date", "")
df.insert(14, "dorm_date", "")

df.insert(15, "service_provider", "")
df.insert(16, "ibcRUC", "")

df["vague"] = False

In [39]:
df.columns.values.tolist()

['node_id',
 'role',
 'name',
 'original_name',
 'former_name',
 'jurisdiction',
 'jurisdiction_description',
 'company_type',
 'status',
 'internal_id',
 'address',
 'incorporation_date',
 'inactivation_date',
 'struck_off_date',
 'dorm_date',
 'service_provider',
 'ibcRUC',
 'countries',
 'country_codes',
 'sourceID',
 'valid_until',
 'note',
 'vague']

In [40]:
columns: list = list(df.columns.values)

columns.remove("address")
columns.insert(9, "address")

columns.remove("status")
columns.insert(14, "status")

columns[17] = "country_codes"
columns[18] = "countries"

columns

['node_id',
 'role',
 'name',
 'original_name',
 'former_name',
 'jurisdiction',
 'jurisdiction_description',
 'company_type',
 'address',
 'internal_id',
 'incorporation_date',
 'inactivation_date',
 'struck_off_date',
 'dorm_date',
 'status',
 'service_provider',
 'ibcRUC',
 'country_codes',
 'countries',
 'sourceID',
 'valid_until',
 'note',
 'vague']

In [41]:
df = df.reindex(columns = columns)
df.head()

,node_id,role,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,...,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note,vague
0,11000001,Intermediary,"MICHAEL PAPAGEORGE, MR.",,,,,,MICHAEL PAPAGEORGE; MR. 106 NICHOLSON STREET B...,10001,...,,ACTIVE,,,ZAF,South Africa,Panama Papers,The Panama Papers data is current through 2015,,False
1,11000002,Intermediary,CORFIDUCIA ANSTALT,,,,,,,10004,...,,ACTIVE,,,LIE,Liechtenstein,Panama Papers,The Panama Papers data is current through 2015,,False
2,11000003,Intermediary,"DAVID, RONALD",,,,,,,10014,...,,SUSPENDED,,,MCO,Monaco,Panama Papers,The Panama Papers data is current through 2015,,False
3,11000004,Intermediary,"DE BOUTSELIS, JEAN-PIERRE",,,,,,,10015,...,,SUSPENDED,,,BEL,Belgium,Panama Papers,The Panama Papers data is current through 2015,,False
4,11000005,Intermediary,THE LEVANT LAWYERS (TLL),,,,,,,10029,...,,ACTIVE,,,LBN,Lebanon,Panama Papers,The Panama Papers data is current through 2015,,False


In [42]:
conn.execute(
    "COPY Entity FROM (LOAD FROM df RETURN *)"
);

### Others

In [43]:
dat_file: pathlib.Path = dat_dir / "nodes-others.csv"
df = pd.read_csv(dat_file, header = 0).astype(str).fillna("")
df.replace({"nan": ""}, regex=True, inplace=True)

df.head()

,node_id,name,type,incorporation_date,struck_off_date,closed_date,jurisdiction,jurisdiction_description,countries,country_codes,sourceID,valid_until,note
0,85004929,ANTAM ENTERPRISES N.V.,LIMITED LIABILITY COMPANY,18-MAY-1983,,28-NOV-2012,AW,Aruba,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,Closed date stands for Cancelled date.
1,85008443,DEVIATION N.V.,LIMITED LIABILITY COMPANY,28-JUN-1989,31-DEC-2002,,AW,Aruba,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,
2,85008517,ARIAZI N.V.,LIMITED LIABILITY COMPANY,19-JUL-1989,,19-MAY-2004,AW,Aruba,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,Closed date stands for Cancelled date.
3,85008542,FLAIRUBA N.V.,LIMITED LIABILITY COMPANY,27-JUL-1989,24-JUL-2000,,AW,Aruba,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,
4,85008583,S.L. ARUBA FISHERIES TRADING N.V.,LIMITED LIABILITY COMPANY,01-AUG-1989,,29-OCT-2007,AW,Aruba,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,Closed date stands for Cancelled date.


In [44]:
df.describe(include = "all").loc[[ "count", "freq", "unique", "top", ]]

,node_id,name,type,incorporation_date,struck_off_date,closed_date,jurisdiction,jurisdiction_description,countries,country_codes,sourceID,valid_until,note
count,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989
freq,1,3,2101,2101,2944,2872,2031,2031,2603,2603,2031,2031,2872
unique,2989,2976,4,835,40,113,7,7,64,64,7,10,3
top,85004929,Shampaign Investments Limited,,,,,,,,,Paradise Papers - Appleby,Appleby data is current through 2014,


Show that the `type` field values are similar to `Entity.company_type`

In [45]:
set(df.type.values)

{'',
 'FOREIGN FORMED CORPORATION',
 'LIMITED LIABILITY COMPANY',
 'SOLE OWNERSHIP'}

Keep track of the set of `node_id` values for this class.

In [46]:
id_other: set = set(df.node_id.values)

Reshape the dataframe to fit our inclusive `Entity` metadata.

In [47]:
df.columns.values.tolist()

['node_id',
 'name',
 'type',
 'incorporation_date',
 'struck_off_date',
 'closed_date',
 'jurisdiction',
 'jurisdiction_description',
 'countries',
 'country_codes',
 'sourceID',
 'valid_until',
 'note']

In [48]:
df.insert(1, "role", "Other")

df.insert(3, "original_name", "")
df.insert(4, "former_name", "")

df.rename(columns = { "type": "company_type" }, inplace = True)

df.insert(6, "address", "")
df.insert(7, "internal_id", "")

df.rename(columns = { "closed_date": "inactivation_date" }, inplace = True)
df.insert(13, "dorm_date", "")
df.insert(14, "status", "")
df.insert(15, "service_provider", "")
df.insert(16, "ibcRUC", "")

df["vague"] = False

In [49]:
columns: list = list(df.columns.values)

columns.remove("jurisdiction")
columns.remove("jurisdiction_description")
columns.insert(5, "jurisdiction")
columns.insert(6, "jurisdiction_description")

columns[11] = "inactivation_date"
columns[12] = "struck_off_date"

columns[17] = "country_codes"
columns[18] = "countries"

columns

['node_id',
 'role',
 'name',
 'original_name',
 'former_name',
 'jurisdiction',
 'jurisdiction_description',
 'company_type',
 'address',
 'internal_id',
 'incorporation_date',
 'inactivation_date',
 'struck_off_date',
 'dorm_date',
 'status',
 'service_provider',
 'ibcRUC',
 'country_codes',
 'countries',
 'sourceID',
 'valid_until',
 'note',
 'vague']

In [50]:
df = df.reindex(columns = columns)
df.head()

,node_id,role,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,...,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note,vague
0,85004929,Other,ANTAM ENTERPRISES N.V.,,,AW,Aruba,LIMITED LIABILITY COMPANY,,,...,,,,,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,Closed date stands for Cancelled date.,False
1,85008443,Other,DEVIATION N.V.,,,AW,Aruba,LIMITED LIABILITY COMPANY,,,...,,,,,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,,False
2,85008517,Other,ARIAZI N.V.,,,AW,Aruba,LIMITED LIABILITY COMPANY,,,...,,,,,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,Closed date stands for Cancelled date.,False
3,85008542,Other,FLAIRUBA N.V.,,,AW,Aruba,LIMITED LIABILITY COMPANY,,,...,,,,,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,,False
4,85008583,Other,S.L. ARUBA FISHERIES TRADING N.V.,,,AW,Aruba,LIMITED LIABILITY COMPANY,,,...,,,,,,,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,Closed date stands for Cancelled date.,False


In [51]:
conn.execute(
    "COPY Entity FROM (LOAD FROM df RETURN *)"
);

In [52]:
results = conn.execute("""
  MATCH (n:Entity)
  RETURN COUNT(*)
""")

while results.has_next():
    ic(results.get_next())

ic| results.get_next(): [1614277]


### Addresses

In [53]:
dat_file: pathlib.Path = dat_dir / "nodes-addresses.csv"

df = pd.read_csv(
    dat_file,
    header = 0,
    low_memory = False,
).astype(str).fillna("")

df.replace({"nan": ""}, regex=True, inplace=True)

df.head()

,node_id,address,name,countries,country_codes,sourceID,valid_until,note
0,24000001,"ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805...",,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,
1,24000002,"SUITE E-2,UNION COURT BUILDING, P.O. BOX N-818...",,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,
2,24000003,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785,...",,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,
3,24000004,"P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O....",,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,
4,24000005,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. ...",,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,


In [54]:
df.describe(include = "all").loc[[ "count", "freq", "unique", "top", ]]

,node_id,address,name,countries,country_codes,sourceID,valid_until,note
count,402246,402246,402246,402246,402246,402246,402246,402246
freq,1,19932,178898,125335,125335,123269,123269,402208
unique,402246,377861,222810,366,226,20,26,12
top,24000001,,,,,Paradise Papers - Malta corporate registry,Malta corporate registry data is current throu...,


Keep track of the set of `node_id` values for this class.

In [55]:
id_address: set = set(df.node_id.values)

In [56]:
df.columns.values.tolist()

['node_id',
 'address',
 'name',
 'countries',
 'country_codes',
 'sourceID',
 'valid_until',
 'note']

In [57]:
conn.execute("""
  CREATE NODE TABLE Address (
    node_id STRING,
    address STRING,
    name STRING,
    countries STRING,
    country_codes STRING,
    sourceID STRING,
    valid_until STRING,
    note STRING,
    PRIMARY KEY (node_id)
  )
""");

In [58]:
conn.execute(
    "COPY Address FROM (LOAD FROM df RETURN *)"
);

In [59]:
results = conn.execute("""
  MATCH (n:Address)
  RETURN *
  LIMIT 1;
""")

while results.has_next():
    ic(results.get_next())

ic| results.get_next(): [{'_id': {'offset': 0, 'table': 1},
                          '_label': 'Address',
                          'address': '7 Mair Mills Dr.; Collingwood; ON; L9Y0A8; Canada',
                          'countries': 'Canada',
                          'country_codes': 'CAN',
                          'name': '',
                          'node_id': '14022522',
                          'note': '',
                          'sourceID': 'Panama Papers',
                          'valid_until': 'The Panama Papers  data is current through 2015'}]


## Connecting entities

### Relationships

In [85]:
dat_file: pathlib.Path = dat_dir / "relationships.csv"

df = pd.read_csv(
    dat_file,
    header = 0,
    low_memory = False,
).astype(str).fillna("")

df.replace({"nan": ""}, regex=True, inplace=True)

df.head()

,node_id_start,node_id_end,rel_type,link,status,start_date,end_date,sourceID
0,10000035,14095990,registered_address,registered address,,,,Panama Papers
1,10000044,14091035,registered_address,registered address,,,,Panama Papers
2,10000055,14095990,registered_address,registered address,,,,Panama Papers
3,10000063,10007006,same_as,same name and registration date as,,,,
4,10000064,14091429,registered_address,registered address,,,,Panama Papers


In [86]:
df.describe(include = "all").loc[[ "count", "freq", "unique", "top", ]]

,node_id_start,node_id_end,rel_type,link,status,start_date,end_date,sourceID
count,3339267,3339267,3339267,3339267,3339267,3339267,3339267,3339267
freq,36373,37338,1720357,589938,3164321,2392562,3070400,776335
unique,1131496,1321323,14,1041,3,24336,13561,12
top,54662,236724,officer_of,shareholder of,,,,Paradise Papers - Malta corporate registry


In [87]:
df.columns.values.tolist()

['node_id_start',
 'node_id_end',
 'rel_type',
 'link',
 'status',
 'start_date',
 'end_date',
 'sourceID']

Double-check: which `node_id` values exist in the `Entity` node table?

In [88]:
results = conn.execute("""
  MATCH (n:Entity)
  RETURN n.node_id;
""")

id_ents: set = set()

while results.has_next():
    row: list = results.get_next()
    id_ents.add(row[0])

len(id_ents)

1614277

Note: which relations are named in ICIJ?

In [89]:
rels: dict = df["rel_type"].value_counts().to_dict()
rels

{'officer_of': 1720357,
 'registered_address': 832721,
 'intermediary_of': 598546,
 'same_name_as': 104170,
 'similar': 46761,
 'same_company_as': 15523,
 'connected_to': 12145,
 'same_as': 4272,
 'same_id_as': 3120,
 'underlying': 1308,
 'similar_company_as': 203,
 'probably_same_officer_as': 132,
 'same_address_as': 5,
 'same_intermediary_as': 4}

This parts gets a bit iffy. There's already been some entity resolution performed, however's this appears to be simply fuzzy string matching on names and addresses. Als, the pre-defined relations are haphazard in terms of domain/range -- and while Neo4j may be forgiving in this aspect of semantic modeling, this artifact may degrade inference downstream.

This is where the _semantic model_ used by ICIJ seems to be messy. It may produce useful visualizations and graph query results; however, good luck trying to use this for inference downstream!

We want to do better. For now we'll skip the questionable relations which were used in the ICIJ/Neo4j collaboration: 

  1. `officer_of`: source nodes with names which are legal placeholders, e.g., "THE BEARER"
  2. `registered_address`: note that some `Entity` nodes have more than one registered address
  3. `intermediary_of`: the domain and range are a jumble of `Entity`, `Officer`, `Intermediary`
  4. `Officer` and `Intermediary` overlap; the former has more information
  5. `same_company_as`: simple string matches, where the domain and range are sometimes mistakingly representing companies `Officer` nodes
  6. `probably_same_officer_as`: simple string matches
  7. (tbd)

#### OfficerOf

Link `Officer` nodes to `Entity` nodes using the `OfficerOf` relation

In [90]:
df[df.rel_type == "officer_of"].head()

,node_id_start,node_id_end,rel_type,link,status,start_date,end_date,sourceID
221749,12000001,10073324,officer_of,shareholder of,,19-NOV-1999,04-JUL-2000,Panama Papers
221751,12000002,10148386,officer_of,shareholder of,,30-MAR-2012,06-JUL-2012,Panama Papers
221753,12000003,10024966,officer_of,shareholder of,,14-JAN-2010,,Panama Papers
221755,12000004,10004763,officer_of,shareholder of,,23-JUL-2012,,Panama Papers
221757,12000005,10206741,officer_of,shareholder of,,13-SEP-2010,,Panama Papers


In [91]:
def check_row (
    node_id_start: int,
    node_id_end: int,
    rel_type: str,
    ) -> bool:
    return rel_type == "officer_of" and node_id_start in id_officer and node_id_end in id_entity

df["todo"] = df.apply(lambda x: check_row(x["node_id_start"], x["node_id_end"], x["rel_type"]), axis = 1)

df_todo = df[df.todo]
del df_todo["rel_type"]
del df_todo["todo"]

df_todo

,node_id_start,node_id_end,link,status,start_date,end_date,sourceID
221749,12000001,10073324,shareholder of,,19-NOV-1999,04-JUL-2000,Panama Papers
221751,12000002,10148386,shareholder of,,30-MAR-2012,06-JUL-2012,Panama Papers
221753,12000003,10024966,shareholder of,,14-JAN-2010,,Panama Papers
221755,12000004,10004763,shareholder of,,23-JUL-2012,,Panama Papers
221757,12000005,10206741,shareholder of,,13-SEP-2010,,Panama Papers
...,...,...,...,...,...,...,...
3339244,240556390,240554206,Beneficiary of trust,,,,
3339245,240556391,240554207,Beneficial owner of the underlying company,,,,
3339246,240556392,240554207,Beneficial owner of the underlying company,,,,
3339247,240556393,240554208,Settlor,,,,


In [66]:
conn.execute("""
  CREATE REL TABLE OfficerOf (FROM Entity TO Entity,
    link STRING,
    status STRING,
    start_date STRING,
    end_date STRING,
    sourceID STRING,
    MANY_MANY
  )
""");

In [92]:
conn.execute(
    "COPY OfficerOf FROM (LOAD FROM df_todo RETURN *)"
);

RuntimeError: Runtime exception: Unable to find primary key value 86017926.

In [94]:
"86017926" in id_officer

True

In [97]:
df_todo[df_todo.node_id_start == "86017926"]

,node_id_start,node_id_end,link,status,start_date,end_date,sourceID
2023779,86017926,85023302,proxy of,,06-APR-2001,,Paradise Papers - Aruba corporate registry


In [98]:
"86017926" in id_ents

True

#### IntermediaryOf

Link `Entity`, `Officer`, `Intermediary` nodes together using the `Intermediary` relation

In [ ]:
df[df.rel_type == "intermediary_of"].head()

In [ ]:
id_entish = id_entity.union(id_officer).union(id_intermed).union(id_other)

def check_row (
    node_id_start: int,
    node_id_end: int,
    rel_type: str,
    ) -> bool:
    return rel_type == "intermediary_of" and node_id_start in id_entish and node_id_end in id_entish

df["todo"] = df.apply(lambda x: check_row(x["node_id_start"], x["node_id_end"], x["rel_type"]), axis = 1)

df_todo = df[df.todo]
del df_todo["rel_type"]
del df_todo["todo"]

df_todo

In [ ]:
conn.execute("""
  CREATE REL TABLE IntermediaryOf (FROM Entity TO Entity,
    link STRING,
    status STRING,
    start_date STRING,
    end_date STRING,
    sourceID STRING,
    MANY_ONE
  )
""");

In [ ]:
conn.execute(
    "COPY IntermediaryOf FROM (LOAD FROM df_todo RETURN *)"
);

#### RegisteredAddress

Link `Entity` nodes to `Address` nodes using the `RegisteredAddress` relation

In [ ]:
df[df.rel_type == "registered_address"].head()

In [ ]:
def check_row (
    node_id_start: int,
    node_id_end: int,
    rel_type: str,
    ) -> bool:
    return rel_type == "registered_address" and node_id_start in id_entish and node_id_end in id_address

df["todo"] = df.apply(lambda x: check_row(x["node_id_start"], x["node_id_end"], x["rel_type"]), axis = 1)

df_todo = df[df.todo]
del df_todo["rel_type"]
del df_todo["todo"]

df_todo

In [ ]:
conn.execute("""
  CREATE REL TABLE RegisteredAddress (FROM Entity TO Address,
    link STRING,
    status STRING,
    start_date STRING,
    end_date STRING,
    sourceID STRING,
    MANY_MANY
  )
""");

In [ ]:
conn.execute(
    "COPY RegisteredAddress FROM (LOAD FROM df_todo RETURN *)"
);